In [5]:
import cv2
import numpy as np
from scipy.spatial import distance
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from torchvision.io import read_video
from re import I
import os
import torch
from torchvision.io import read_video
from IPython.display import HTML, display
from base64 import b64encode
from PIL import Image
import io
from sklearn.model_selection import train_test_split

In [2]:
def extract_frames(video_path, every_n_frames=1):
    """Extracts frames from a video file."""
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    for i in range(frame_count):
        ret, frame = cap.read()
        if i % every_n_frames == 0 and ret:
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    cap.release()
    return frames

# Create a display handle
display_handle = display(display_id=True)

In [3]:
root_dir = r'C:\Shay\VIZ.AI\cardiac_stage_pred\EchoNet-Dynamic'

In [6]:
df_tracing = pd.read_csv(os.path.join(root_dir, 'VolumeTracings.csv'))

def order_coordinates(coords):
    coords = list(coords)
    polygon = [coords.pop(0)]  # Start with the first point
    while coords:
        last_point = polygon[-1]
        distances = distance.cdist([last_point], coords, 'euclidean')[0]
        nearest_index = np.argmin(distances)
        polygon.append(coords.pop(nearest_index))
    return polygon

def process_group(group):
    combined_coords = group['r1'].tolist() + group['r2'].tolist()
    ordered_polygon = order_coordinates(combined_coords)
    return np.array(ordered_polygon).round().astype(np.int32)

df_tracing['r1'] = list(zip(df_tracing['X1'], df_tracing['Y1']))
df_tracing['r2'] = list(zip(df_tracing['X2'], df_tracing['Y2']))

# Group by 'FileName' and 'Frame'
gb_tracing = df_tracing.groupby(['FileName', 'Frame'])

# Apply the function to each group
df_polygons = gb_tracing.apply(process_group).reset_index(name='polygon')

# Save the polygons
df_polygons.to_csv(os.path.join(root_dir, 'df_polygons.csv'), index=False)

C:\Users\shay2\AppData\Local\Temp\ipykernel_9284\3334704470.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_polygons = gb_tracing.apply(process_group).reset_index(name='polygon')


In [ ]:
def update_frame(change,frame_cords,frames, df_polygons, filename):
    frame_idx = change['new']
    zoom_level = 2
    # Check if there's a line to draw for the current frame
    if frame_idx in frame_cords['Frame'].values:
        # Find the row in the DataFrame for the current frame
        frame_rows = frame_cords[frame_cords['Frame'] == frame_idx]
        # Extract line coordinates
        # Draw the line on the frame
        frame_with_lines = frames[frame_idx].copy()
        for _, row in frame_rows.iterrows():
          x1, y1, x2, y2 = round(row['X1']), round(row['Y1']), round(row['X2']), round(row['Y2'])
          slope = (y2 - y1) / (x2 - x1) if x2 != x1 else float('inf')  # Avoid division by zero

            # Choose the color based on the slope
          color = (255, 0, 0)  # Default color (red) for non-positive slope
          if slope > 0:
              color = (0, 0, 255)  # Blue color for positive slope

          patient_index = df_polygons.index[(df_polygons['FileName'] == filename) & (df_polygons['Frame'] == frame_idx)].tolist()[0]
          pts = df_polygons.iloc[patient_index]['polygon']
          # pts = np.concatenate((pts[:pts.shape[0]//2], pts[:pts.shape[0]//2 - pts.shape[0]:-1]))
          # pts = np.concatenate([pts, pts[[0]]])
          # cv2.polylines(frame_with_lines, [pts], True, (0,255,255))
          cv2.fillPoly(frame_with_lines, [pts], 255)
          # print(pts.shape)
    else:
        frame_with_lines = frames[frame_idx]

    # Resize the frame
    frame_with_lines = cv2.resize(frame_with_lines, None, fx=4, fy=4, interpolation=cv2.INTER_LINEAR)

    # Encode the frame to display in the notebook
    _, encoded_image = cv2.imencode('.png', frame_with_lines)
    encoded_image = b64encode(encoded_image).decode('utf-8')
    image_html = f'<img src="data:image/png;base64,{encoded_image}" style="width: auto; height: auto;">'

    # Update the display with the new image
    display_handle.update(HTML(image_html))

In [ ]:
def analyzeImage(video_path, df_polygons, filename):
  df = df_tracing[df_tracing['FileName'] ==  filename ]
  print(f'Processing image{filename}')
  print(f'This video has labels from frames: { df.Frame.unique()}')
  frames = extract_frames(video_path, every_n_frames=1)
  frame_slider = widgets.IntSlider(value=df.Frame.unique()[0], min=0, max=len(frames)-1, description='Frame:')
  zoom_slider = widgets.FloatSlider(value=1.0, min=0.5, max=4.0, step=0.1, description='Zoom:')

  frame_slider.observe(lambda change: update_frame(change, df, frames, df_polygons, filename), names='value')
  zoom_slider.observe(lambda change: update_frame(change, df,frames, df_polygons, filename), names='value')


  # Display the widgets
  display(widgets.VBox([frame_slider, zoom_slider]))

  # Initial display to show the first frame
  update_frame({'new': df.Frame.unique()[0]}, df,frames, df_polygons, filename)  # Simulate change dict if needed



In [ ]:
filename = os.listdir(os.path.join(root_dir, 'Videos'))[7285]

video_path = os.path.join(root_dir, 'Videos', filename)


In [9]:
print(filename)
analyzeImage(video_path, df_polygons, filename)

0X66DC4A8DD37055CC.avi
Processing image0X66DC4A8DD37055CC.avi
This video has labels from frames: [50 66]


In [13]:
print(df_polygons['polygon'][0].shape)

(42, 2)
